# Excercises 

## 0. Setup your own repo
- Setting up a repo is often repetitive. That's why you can use what are called 'cookiecutters', providing you with a template structure with some basic folders and files already set up for you. You don't have to use one, and can do it by hand, but have a look and maybe you think it is helpful.
    - You could use the `cookiecutter` command that is preinstalled on your VM to create a repo, see https://github.com/raoulg/datascience-cookiecutter for details. I made this one myself, because I ended up editing the cookiecutter I was using.
    - Another project is this one https://github.com/drivendata/cookiecutter-data-science , also intended for data science projects
    - more general cookiecutters are shipped with tools like [pdm](https://pdm-project.org/latest/) and [rye](https://rye-up.com/); starting a project with `pdm init` or `rye init` (see docs for details) will provide you with some minimal structure, and there is the option to provide your own template with pdm (see [pdm template](https://pdm-project.org/latest/usage/template/))
- push your own repo to github. Use `MADS-ML-{yourname}` as a format, eg `MADS-ML-JoostB`.
- Invite me (raoulg; https://github.com/raoulg) as a collaborator to your repo.
- make the excercises 1-5 below in your repo, and push them to github.

Tips:
- Commit often (every 30 minutes or so) 
- really, commit often. commiting and pushing your work is the best way to make sure your work is saved properly.
- Commit groups of files that are related to each other. If you have more files, commit them separately.
- Write commit messages that are descriptive and informative. "lesson 1", "changes" or "commit" are bad commit messages; "added excercise 2" is better, "[exercise 2] added __len__ to Dataset class" is even better.
- Use `pdm` or `rye` to add dependencies. `mads_datasets` and `mltrainer` should cover a lot of what you need; don't blindly copy-paste all dependencies but keep your `pyproject.toml` as clean as possible.

At some point, you will get a grade for the excercises that is 0 (not good enough), 1 (good enough) or 2 (excellent).
I will look for both form and correctness to determine your grade.
The result be incorporated into your final grade for this course.

## 1. 3D Tensor dataset
- Create a random 3D tensor dataset with `torch`
- Build your own `DataSet` class, such that you can get a 3D tensor and a label (which can be a random 0 or 1)
See notebook 03_dataloader for details on how to create a custom dataset. See 01_tensors and the torch documentation how to create random tensors.

## 2. Datastreamers
Study the `BaseDatastreamer` in `03_dataloader` and use it with your own dataset, such that you get a datastreamer that will keep on giving you new batches of data when you call `next()` or loop over it.

# 3. Tune the network
For this exercise we won't build upon the previous exercises, but instead will use the Fashion dataset.
Run the experiment below, explore the different parameters (see suggestions below) and study the result with tensorboard. 

In [1]:
from mads_datasets import DatasetFactoryProvider, DatasetType

from mltrainer.preprocessors import BasePreprocessor
from mltrainer import imagemodels, Trainer, TrainerSettings, ReportTypes, metrics

import torch.optim as optim
import gin

In [2]:
gin.parse_config_file("model.gin")

/home/azureuser/MachineLearning/.venv/lib/python3.11/site-packages/gin/config.py:615: FutureWarning: `NLLLoss2d` has been deprecated. Please use `NLLLoss` instead as a drop-in replacement and see https://pytorch.org/docs/main/nn.html#torch.nn.NLLLoss for more details.
  decorated_class = decorating_meta(cls.__name__, (cls,), overrides)


ParsedConfigFileIncludesAndImports(filename='model.gin', imports=['gin.torch.external_configurables'], includes=[])